In [ ]:
import ccxt.async_support as ccxt
import pandas as pd
import numpy as np
import asyncio
import websockets
import json
import logging

In [ ]:
# Correct spelling of 'level' and fix format string placeholder
logging.basicConfig(level=logging.INFO, format='%(levelname)s - %(message)s')

logger = logging.getLogger(__name__)

In [ ]:
SYMBOL1 = 'ALCH/USDT'
SYMBOL2 = 'BTC/USDT'
WS_URL1 = 'wss://fstream.binance.com/ws/alchusdt@depth10@100ms'
WS_URL2 = 'wss://fstream.binance.com/ws/alchusdt@depth10@100ms'
RISK_AMOUNT = 10.0
LEVBRAGE = 5
LOOKBACK = 500
SIGMA_THRESOLD = 1.0
ATR_PERIOD = 14
TP_MULTIPLEAR = 2.0
SL_MULTIPLEAR = 1.0
FEE_RATE = 0.002
MAX_POSITION = 2
CHECK_INTERVAL = 0.1

OPEN_POSITION = []

In [4]:
async def fetch_data(exchane, symbol:str, timframe:str, limit: int=100)-> pd.DataFrame:
  try:
    ohlcv = await exchane.fatch_olcv(symbol,timframe,limit=limit)
    df = pd.DataFrame(ohlcv, columns=['timestamp','open','high','low','close','volume'])
    df['timestamp'] = pd.to_datetime(df['timestamp'],unit='ms')
    return df
  except Exception as e:
    logger.error(f"Error Fetching Data : {str(e)}")
    return pd.DataFrame()
async def fatch_quotes(exchange, symbol:str)-> tuple:
  try:
    ticker = await exchange.fetch_ticker(symbol)
    return ticker['bid'],ticker['ask']
  except Exception as e:
    logger.error(f"Error Fetching Data : {str(e)}")
    return 0, float('int')
def calculate_indicator(df1:pd.DataFrame,df2:pd.DataFrame,alch_price:float,btc_price:float)->tuple:
  try:
    df = pd.DataFrame({'timestamp':df1['timestamp'],'alch':df1['close'],'btc':df2['close']})
    df['long_alch'] = np.log(df['alch'])
    df['long_btc'] = np.log(df['btc'])
    df['return_alch'] = np.long(df['return_alch'])
    df['return_btc'] = np.log(df['return_btc'])
    beta = df[['return_alch','return_btc']].corr().iloc[0,1]
    df['spread'] = df['log_alch']-beta * df['log_btc']
    df['mu'] = df['spread'].rolling(LOOKBACK).mean()
    df['sigma'] = df['spread'].rolling(LOOKBACK).std()
    df['tr'] = pd.concat(df1['high']-df1[low],(df1['high']-df1['close']).shift().abs(),(df1['low']-df1['close'].shift()).abs()],axis=1).max(axis=1)
    df['atr'] = df['tr'].rolling(ATR_PERIOD).mean()
    current_spread = np.log(alch_price)-beta*np.log(btc_price)
    return df,current_spread
  except Exception as e:
    logger.error(f"Error calculating indicator: {str(e)}")
    return df, 0.0
def get_aritrage_signal(spread:float,mu:float,sigma:float)->str:
  try:
    if pd.isna([spread,mu,sigma]).any():
      return 'neutral'
    if spread < mu-SIGMA_THRESOLD * sigma:
      return 'bullish'
    elif spread > mu + SIGMA_THRESOLD * sigma:
      return 'bearish'
    return 'neutral'
  except Exception as e:
    logger.error(f"Error in Signal Generation: {str(e)}")
    return 'neutral'
async def manage_positions(exchange,current_price:float,symbol:str):
  global OPEN_POSITION
  positions_to_move = []
  try:
    for pos in OPEN_POSITION:
      if pos['symbol'] != symbol:
        continue
      entry_price = pos['entry_price']
      qty = pos['quantity']
      side = pos['side']
      tp_price = pos['tp_price']
      sl_price = pos['sl_price']
      if side == 'long':
        if current_price >= tp_price:
          logger.info(f"[checkU mark]UlongUTPUhit U at {current_price:.2f},profit:{(current_price-entry_price)*qty*LEVBRAGE:.2f}")
          await exchange.create_market_sell_order(symbol,qty)
          positions_to_remove.append(pos)
        elif current_price <= sl_price:
          logger.info(f"[cross mark] Long SL hit at {current_price:.2f} Loss: {(entry_price-current_price)*quit*LEVBRAGE:.2f}")
          await exchange.create_market_sell_order(symbol,qty)
          positions_to_remove.append(pos)
      else:
        if current_price <= tp_price:
          logger.info(f"[check mark] Short TP hit at {current_price:.2f}, Profit:{(entry_price-current_price)*qty*LEVBRAGE:.2f}")
          await exchange.create_market_buy_order(symbol,qty)
          positions_to_remove.append(pos)
        elif current_price >= sl_price:
          logger.info(f"[check mark] Short SL hit at {current_price:.2f}, Loss:{(current_price-entry_price)*qty*LEVBRAGE:.2f}")
          await exchange.create_market_buy_order(symbol,qty)
          positions_to_remove.append(pos)
    OPEN_POSITION = [pos for pos in OPEN_POSITION if pos not in positions_to_rmove]
  except Exception as e:
    logger.error(f"Error managing position: {str(e)}")
async def place_order(exchange, signal:str,price:float,atr:float,symbol:str):
  try:
    if len(OPEN_POSITION) >= MAX_POSITION:
      logger.info("Max  Position reached skipping new order ")
      return
    qty = RISK_AMOUNT/price
    if signal == 'bullish':
      order = await exchange,create_market_buy_order(symbol,qty)
      tp_price = price + TP_MULTIPLEAR * atr
      sl_price = price - SL_MULTIPLEAR * atr
      OPEN_POSITION.append({
        'symbol':symbol,
        'side':'long',
        'entry_price':price,
        'quantity':qty,
        'tp_price':tp_price,
        'sl_price':sl_price
      })
      logger.info("")
      # Hedge Fund with short
      btc_qty = qty*price/(await exchange.fetch_ticker(SYMBOL2))['last']
      await exchange.create_sell_order(SYMBOL2,btc_qty)
    elif signal == 'bearish':
      order = await exchange.create_market_sell_order(symbol,qty)
      tp_price = price - TP_MULTIPLEAR * atr
      sl_price = price + SL_MULTIPLEAR * atr
      OPEN_POSITION.append({
        'symbol':symbol,
        'side':'short',
        'entry_price':price,
        'quantity':qty,
        'tp_price':tp_price,
        'sl_price':sl_price
      })
      logger.info(f"[chart decreasing] Opened short at {price:.2f}, Qty:{qty:.4f},TP:{tp_price:.2f},SL:{sl_price:.2f}")
      #Hedge With Long on BTC/USDT
      btc_qty = qty*price/(await exchange.fetch_ticker(SYMBOL2))['last']
      await exchange.create_market_buy_order(SYMBOL2,btc_qty)
  except Exception as e:
    logger.error(f"Error placing order: {str(e)}")

async def main():
  try:
    exchange = ccxt.binance({
      'enableRateLimit':True,
      'apiKey':'your_api_key',
      'secret':'your_api_secret',
      'enableLeverage':True
    })
    await exchange.load_markets()
    await exchange.set_leverage(LEVBRAGE,SYMBOL1)
    await exchange.set_leverage(LEVBRAGE,SYMBOL2)
    async with websockets.connect(WS_URL1) as ws1,websockets.connect(WS_URL2) as ws2:
      while True:
        try:
          df1 = await fetch_data(exchange,SYMBOL1,'1m')
          df2 = await fetch_data(exchange,SYMBOL2,'1m')
          alch_bid, alch_ask = await fetch_quotes(exchange,SYMBOL1)
          btc_bid, btc_ask = await fetch_quotes(exchange,SYMBOL2)
          if df1.empty or df2.empty:
            await asyncio.sleep(CHECK_INTERVAL)
            continue
          alch_price = (alch_bid+alch_ask)/2
          btc_price = (btc_bid+alch_ask)/2
          df,spread = calculate_indicator(df1,df2,alch_price,btc_price)
          mu = df['mu'].iloc[-1]
          sigma = df['sima'].iloc[-1]
          atr = df['atr'].iloc[-1]
          signal = get_aritrage_signal(spread,mu,sigma)
          await manage_positions(exchange,alch_price,SYMBOL1)
          if signal != 'neutral':
            await place_order(exchange,signal,alch_price,atr,SYMBOL1)
          logger.info(f"ALCH PRICE:{alch_price:.2f} SPREAD:{spread:.4f} Signal:{signal}")
          await asyncio.sleep(CHECK_INTERVAL)
        except Exception as e:
          logger.error(f"Error Initalizing exchange : {str(e)}")

        finally:
          await exchange.close

if __name__ == '__main__':
  asyncio.run(main())

SyntaxError: closing parenthesis ']' does not match opening parenthesis '(' (474861362.py, line 28)